# Reconhecimento de Folhas com Machine Learning

An introduction to machine learning with scikit-learn
https://scikit-learn.org/stable/
https://scikit-learn.org/stable/tutorial/basic/tutorial.html

Supervised learning
https://scikit-learn.org/stable/supervised_learning.html#supervised-learning

Unsupervised learning
https://scikit-learn.org/stable/unsupervised_learning.html

Neste notebook você concluirá seu primeiro projeto de Machine Learning usando Python. 

Um projeto de Machine Learning tem uma série de etapas bem conhecidas:

- Defina o problema.
- Prepare os dados.
- Avalie algoritmos.
- Melhore os resultados.
- Resultados finais.

![image.png](attachment:14b85f78-7ab2-4425-9a9e-d3bb08480ce5.png)

# Treinamento Supervisionado para Modelar

![image.png](attachment:53109a22-6bcc-4eaf-bc75-3e4dcef808ad.png)

* Import das bibliotecas

In [1]:
import logging
import pandas as pd
import pandas_profiling
import pickle as pkl
import sys
import wandb

from sklearn.model_selection import train_test_split

Instalação pandas-profiling
```python
!{sys.executable} -m pip install pandas-profiling

!jupyter nbextension enable --py widgetsnbextension
```

Login no Wandb
```python
!wandb login
```

Setando o logger

In [2]:
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

Instanciando o wandb

In [3]:
run = wandb.init(project="eldorado_leaf", group="eda", save_code=True)

wandb: Currently logged in as: saraselis (use `wandb login --relogin` to force relogin)


# Carregar o conjunto de dados
Podemos carregar diretamente do aquivo .csv.

Tudo deve carregar sem erros. Se você tiver um erro, pare. Instale as libs que ainda não estão instaladas, usando pip install <lib_name>

## Leaf Dataset

The provided data comprises the following shape (attributes 3 to 9) and texture (attributes 10
to 16) features:

1. Class (Species)
2. Specimen Number
3. Eccentricity
4. Aspect Ratio
5. Elongation
6. Solidity
7. Stochastic Convexity
8. Isoperimetric Factor
9. Maximal Indentation Depth
10. Lobedness
11. Average Intensity
12. Average Contrast
13. Smoothness
14. Third moment
15. Uniformity
16. Entropy

Como o dataset vem sem as colunas, vamos setar as colunas

In [4]:
names = ['Class', 'Specimen Number', 'Eccentricity', 'Aspect Ratio', 'Elongation', 'Solidity', 'Stochastic Convexity', 'Isoperimetric Factor',
         'Maximal Indentation Depth', 'Lobedness', 'Average Intensity', 'Average Contrast', 'Smoothness', 'Third moment',
         'Uniformity', 'Entropy']

Leitura do dataset

In [5]:
data_leaf = pd.read_csv('leaf.csv', names=names)

Dataset

In [6]:
data_leaf.sample()

,Class,Specimen Number,Eccentricity,Aspect Ratio,Elongation,Solidity,Stochastic Convexity,Isoperimetric Factor,Maximal Indentation Depth,Lobedness,Average Intensity,Average Contrast,Smoothness,Third moment,Uniformity,Entropy
38,4,7,0.39289,1.1286,0.17039,0.96405,1.0,0.79407,0.011761,0.025176,0.025409,0.078211,0.00608,0.00153,0.000158,1.0326


Salvando o dataset processado no wandb

In [7]:
artifact = wandb.Artifact(
        'data_leaf_processado',
        type='csv',
        description='Dataset após o processamento das colunas')

artifact.add_file('leaf.csv')

<ManifestEntry digest: 8enho+coZZ231SjBNt9l1g==>

In [8]:
run.log_artifact(artifact)

In [9]:
data_leaf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Class                      340 non-null    int64  
 1   Specimen Number            340 non-null    int64  
 2   Eccentricity               340 non-null    float64
 3   Aspect Ratio               340 non-null    float64
 4   Elongation                 340 non-null    float64
 5   Solidity                   340 non-null    float64
 6   Stochastic Convexity       340 non-null    float64
 7   Isoperimetric Factor       340 non-null    float64
 8   Maximal Indentation Depth  340 non-null    float64
 9   Lobedness                  340 non-null    float64
 10  Average Intensity          340 non-null    float64
 11  Average Contrast           340 non-null    float64
 12  Smoothness                 340 non-null    float64
 13  Third moment               340 non-null    float64

Relatório do pandas sobre o dataset

In [10]:
profile = pandas_profiling.ProfileReport(data_leaf)
profile.to_widgets()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
# Data shape
logging.info(f'Dataset features names: {data_leaf.shape}')

INFO:root:Dataset features names: (340, 16)
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.wandb.ai:443
DEBUG:urllib3.connectionpool:https://api.wandb.ai:443 "POST /graphql HTTP/1.1" 200 None


## Classes

In [12]:
# Classes (Labels)
y = data_leaf['Class']

# Classe distribuição


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.wandb.ai:443
DEBUG:urllib3.connectionpool:https://api.wandb.ai:443 "POST /graphql HTTP/1.1" 200 None


In [13]:
y

0       1
1       1
2       1
3       1
4       1
       ..
335    36
336    36
337    36
338    36
339    36
Name: Class, Length: 340, dtype: int64

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.wandb.ai:443
DEBUG:urllib3.connectionpool:https://api.wandb.ai:443 "POST /graphql HTTP/1.1" 200 None


In [14]:
# Classe shape
logging.info(y.shape)

# Classe types
logging.info(type(y))

INFO:root:(340,)
INFO:root:<class 'pandas.core.series.Series'>
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.wandb.ai:443
DEBUG:urllib3.connectionpool:https://api.wandb.ai:443 "POST /graphql HTTP/1.1" 200 None


## Entradas

In [16]:
# Entrada (input)
X = data_leaf.drop('Class', axis=1)
X.head()

,Specimen Number,Eccentricity,Aspect Ratio,Elongation,Solidity,Stochastic Convexity,Isoperimetric Factor,Maximal Indentation Depth,Lobedness,Average Intensity,Average Contrast,Smoothness,Third moment,Uniformity,Entropy
0,1,0.72694,1.4742,0.32396,0.98535,1.00000,0.83592,0.004657,0.003947,0.047790,0.127950,0.016108,0.005232,0.000275,1.17560
1,2,0.74173,1.5257,0.36116,0.98152,0.99825,0.79867,0.005242,0.005002,0.024160,0.090476,0.008119,0.002708,0.000075,0.69659
2,3,0.76722,1.5725,0.38998,0.97755,1.00000,0.80812,0.007457,0.010121,0.011897,0.057445,0.003289,0.000921,0.000038,0.44348
3,4,0.73797,1.4597,0.35376,0.97566,1.00000,0.81697,0.006877,0.008607,0.015950,0.065491,0.004271,0.001154,0.000066,0.58785
4,5,0.82301,1.7707,0.44462,0.97698,1.00000,0.75493,0.007428,0.010042,0.007938,0.045339,0.002051,0.000560,0.000024,0.34214


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.wandb.ai:443
DEBUG:urllib3.connectionpool:https://api.wandb.ai:443 "POST /graphql HTTP/1.1" 200 None


In [17]:
# Entrada shape
logging.info(X.shape)

# Entrada types
logging.info(type(X))

INFO:root:(340, 15)
INFO:root:<class 'pandas.core.frame.DataFrame'>
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.wandb.ai:443
DEBUG:urllib3.connectionpool:https://api.wandb.ai:443 "POST /graphql HTTP/1.1" 200 None


## Separa os dados de treinamento e teste

In [ ]:
#

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=None, random_state=0)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.wandb.ai:443


In [43]:
logging.info(X_train.shape)
logging.info(y_train.shape)
logging.info(X_test.shape)
logging.info(y_test.shape)

INFO:root:(255, 15)
INFO:root:(255,)
INFO:root:(85, 15)
INFO:root:(85,)


In [46]:
pkl.dump(X_train, open('model/Xy/X_train.pkl', 'wb'))
pkl.dump(y_train, open('model/Xy/y_train.pkl', 'wb'))

pkl.dump(X_test, open('model/Xy/X_test.pkl', 'wb'))
pkl.dump(y_test, open('model/Xy/y_test.pkl', 'wb'))

## Treinar o modelo de Machine Learning

In [ ]:
# Load Libs

import sklearn
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import set_config

In [ ]:


pipe = Pipeline(steps)

# O pipeline pode ser usado com qualquer outro classificador
# evita o vazamento do conjunto de teste no conjunto de treinamento

model_deploy = pipe.fit(X_train, y_train)

In [ ]:
print('Training set score: ' + str(model_deploy.score(X_train,y_train)))
print('Test set score: ' + str(model_deploy.score(X_test,y_test)))

In [ ]:
set_config(display='diagram')
model_deploy  # clique no diagrama abaixo para ver os detalhes de cada etapa

## Salvar o modelo de Machine Learning para disponibilizar na Cloud IBM

In [ ]:
# save the model to disk
pkl.dump(model_deploy, open('model_leaf_deploy.pkl', 'wb'))

# Para melhorar as métricas de classificação do seu primeiro modelo, consulte o material da aula 05